In [ ]:
import visa

In [ ]:
rm = visa.ResourceManager()

In [ ]:
rm.list_resources()

In [ ]:
inst = rm.open_resource('ASRL2::INSTR')

In [ ]:
inst.query('*IDN?')

In [ ]:
inst2 = rm.open_resource('ASRL1::INSTR')

In [ ]:
inst2.query('*IDN?')

In [ ]:
import usb

the following code is motivated by [this post](https://sourceforge.net/p/pyusb/mailman/message/26074876/)

In [ ]:
[(j.manufacturer,j.product,j.idProduct,j.idVendor,j.bcdDevice) for j in usb.core.find(find_all=True)]

In [ ]:
[dir(j) for j in usb.core.find(find_all=True)]

In [ ]:
afg = [j for j in usb.core.find(find_all=True) if j.product==u'AFG-2225'][0]

In [ ]:
afg_dev = [l for l in [[k for k in j.devices if k.idProduct==28 and k.idVendor==8580] for j in usb.busses()] if len(l)>0][0][0]

In [ ]:
for j in usb.busses():
    for dev in j.devices:
        if dev.idVendor == 8580 and dev.idProduct==28:
            handle=dev.open()
            handle.setConfiguration(1)
            handle.setAltInterface(0)
            handle.interruptWrite(2, "*IDN?\n")
            handle.releaseInterface()

In [ ]:
hand=afg_dev.open()
print hand
print dir(hand)
hand.reset()
interf = afg_dev.configurations[0].interfaces[0][0]

In [ ]:
hand.claimInterface(interf)
ID='\x01\x01\xfe\x00\x05\x00\x00\x00\x01\x00\x00\x00*idn?\x00\x00\x00'
REQ_IN='\x02\x02\xfd\x00\x00\x01\x00\x00\x00\x00\x00\x00'
hand.bulkWrite(6,ID,4000)    #write to a bulk-out endpoint
hand.bulkWrite(6,REQ_IN,4000)    #write to a bulk-out endpoint
stuff=hand.bulkRead(133,256)#read from a bulk-in endpoint

In [ ]:
from serial.tools.list_ports import comports
import serial

In [ ]:
for j in comports():
    print j[0]
    if 'A105' in j[0]:
        with serial.Serial(j[0]) as s:
            s.write('*idn?\n')
            print s.readline()

In [ ]:
import usb.core

import usb.util

dev = usb.core.find(idVendor=8580, idProduct=28)
if dev is None:
    raise ValueError('Device not found')
    
dev.set_configuration()
cfg = dev.get_active_configuration()
intf = cfg[(0,0)]
#print intf
for j in intf.endpoints():
    j.write('*idn?\n',3)
    #j.read(100)
    #print j
    
    
#usb.util.find_descriptor?

In [ ]:
    intf,
    custom_match = \
    lambda e: \
    usb.util.endpoint_direction(e.bEndpointAddress) == \
    usb.util.ENDPOINT_OUT)

assert ep is not None

ep.write('test')